In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# House Price Prediction

In the projection we would be predicting the prices for the houses based on different features. Every buyer has a different requirements about their dream house and in this project we would analyse how each attribute contribute to the house pricing and the trends by year.

## Goals of the Study

- Predict the House Price based on the attributes 
- The Current data has following properties:
    - `79 features` with `Numerical` and `categorical` columns 
    - `1460` records in the dataset.
    - `SalePrice` is the Label to be predicted
    - The Sales data are from `2006` to `2010`


## 1. Data Preprocessing

#### 1.1 Import Libraries and configuration

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.api.types import CategoricalDtype

from category_encoders import MEstimateEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_regression
import os
sns.set(style="whitegrid", color_codes=True)
sns.set(font_scale=1)
'''Ignore deprecation and future, and user warnings.'''
import warnings as wrn
wrn.filterwarnings('ignore', category = DeprecationWarning) 
wrn.filterwarnings('ignore', category = FutureWarning) 
wrn.filterwarnings('ignore', category = UserWarning) 
plt.rcParams["figure.figsize"] = (10,5)

house_attributes = dict()
house_mapping = dict()

file_name = {"train": "/kaggle/input/house-prices-advanced-regression-techniques/train.csv", 
             "test": "/kaggle/input/house-prices-advanced-regression-techniques/test.csv"}
dict_file_name = "/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt"

In [3]:
from IPython.core.display import HTML

# Reading the mapping file
def load_dictionary(data_dictory):
    '''Read the data dictionary and create a Map'''
    key, value = None, None

    with open(data_dictory) as file:
        lines = file.readlines()
        for line in lines:
            if len(line.strip()) > 0:
                if len(line[0].strip()) > 0:
                    key, value = line.split(":")
                    house_attributes[key.strip()] = value.strip()
                    house_mapping[key] = dict()
                elif key:
                    attr, *attr_value = line.strip().split("\t")
                    house_mapping[key][attr] = " ".join(attr_value)
    return house_mapping, house_attributes



In [4]:
house_mapping,house_attributes = load_dictionary(dict_file_name)
for key in house_attributes:
    print(f"""{key} : {house_attributes[key]}
    ---------------------------------------------
    {house_mapping[key]}
    """)

### 1.2 Reading the House Price data

In [5]:
houseprice = pd.read_csv(file_name['train'], index_col=0)
houseprice_test = pd.read_csv(file_name['test'], index_col=0)

In [6]:
numerical_columns = houseprice.select_dtypes(include=np.number)
categorical_columns = houseprice.select_dtypes(include=['object'])
print(f"""Listing the Columns({len(houseprice.columns)} columns):

Numerical columns({len(numerical_columns)}) :  {numerical_columns.columns.tolist()}

Categorical columns({len(categorical_columns)}) :  {categorical_columns.columns.tolist()}

""")

### 1.3 Summarize the data

In [7]:
print(f"The dataset has data for {houseprice.shape[0]} transaction and columns {houseprice.shape[1]}")
print("Number of dupicated records in the dataset :",len(houseprice[houseprice.duplicated()]))
pd.options.display.float_format = '{:20.2f}'.format
print("Displaying the first 5 records of data:")
houseprice.head(n=5)

#### 1.3.1 Label - Sales Price 
Let’s get statistical information about the label

In [8]:
stat_saleprice = houseprice.SalePrice.describe()
print(f"""Statistics for the SalesPrice:
---------------------------
count    : {stat_saleprice['count']},
mean     : ${stat_saleprice['mean']},
std      : ${stat_saleprice['std']},
min      : ${stat_saleprice['min']},
25%      : ${stat_saleprice['25%']},
50%      : ${stat_saleprice['50%']},
75%      : ${stat_saleprice['75%']},
max      : ${stat_saleprice['max']},
IQR      : ${stat_saleprice['25%']} - ${stat_saleprice['75%']}
skew     : {houseprice.SalePrice.skew()}
skew(log): {np.log(houseprice.SalePrice).skew()}
kurt     : {houseprice.SalePrice.kurt()}
kurt(log): {np.log(houseprice.SalePrice).kurt()}
""")

#### 1.3.2 Numerical columns  
Let’s get statistical information about the Numerical Columns

In [9]:
numerical_columns = houseprice.select_dtypes(include=[np.number]).drop(columns=['SalePrice']).columns.tolist()
uniqueValCount=houseprice[numerical_columns].nunique()
#uniqueValCount[uniqueValCount<50].sort_values(ascending=False).index
numerical_discrete=uniqueValCount[uniqueValCount<50].index.tolist()
time_columns = ['YearBuilt','YearRemodAdd','YrSold','MoSold','GarageYrBlt']
numerical_discrete = [col for col in numerical_discrete if col not in time_columns]
numerical_columns = [col for col in numerical_columns if col not in numerical_discrete+time_columns]

In [10]:
def data_range_nooutlier(data, level=1, continuous=False, log=False):
    def pct_method(data, level):
        # The percentile method cuts off a predefined percentage(99%)
        # amount from the top and the bottom of a distribution
        upper = np.percentile(data, 100 - level)
        lower = np.percentile(data, level)
        # Returning the upper and lower limits
        return [lower, upper]
    
    def iqr_method(data):
        # The interquartile range approach first calculates the interquartile range (IQR) of the data.
        # This IQR is then multiplied with 1.5. Any data that is then further away than the 75 percentile
        # plus 1.5*IQR or 25 percentile minus 1.5*IQR is classified as an outlier.
        perc_75 = np.percentile(data, 75)
        perc_25 = np.percentile(data, 25)
        iqr_range = perc_75 - perc_25
        # Obtaining the lower and upper bound
        iqr_upper = perc_75 + 1.5 * iqr_range
        iqr_lower = perc_25 - 1.5 * iqr_range
        # Returning the upper and lower limits
        return [iqr_lower, iqr_upper]
    
    def std_method(data):
        # given a normally distributed variable, approximately 99.7%
        # of the data is within three standard deviations
        std = np.std(data)
        upper_3std = np.mean(data) + 3 * std
        lower_3std = np.mean(data) - 3 * std
        # Returning the upper and lower limits
        return [lower_3std, upper_3std]
    
    # Taking logs is specified
    data = data[~data.isna()]

    if log is True:
        data = np.log1p(data)
    # Obtaining the ranges
    pct_range = pct_method(data, level)
    iqr_range = iqr_method(data)
    std_range = std_method(data)

    if continuous is False:
        # Setting the lower limit fixed for discrete variables
        low_limit = np.min(data)
        high_limit = np.max([pct_range[1], iqr_range[1], std_range[1]])
    elif continuous is True:
        low_limit = np.min([pct_range[0], iqr_range[0], std_range[0]])
        high_limit = np.max([pct_range[1], iqr_range[1], std_range[1]])
    # Restrict the data with the minimum and maximum
    # outlier = data.between(low_limit, high_limit)
    # Return boolean
    return low_limit, high_limit

In [11]:
numerical_columns_summary = houseprice.loc[:,numerical_columns].describe().T
numerical_columns_summary['skew']=numerical_columns_summary.index.map(lambda x: houseprice[x].skew() )
numerical_columns_summary['skew(log1p)']=numerical_columns_summary.index.map(lambda x: np.log1p(houseprice[x]).skew() )
numerical_columns_summary['cutoffmax']=numerical_columns_summary.index.map(lambda x: data_range_nooutlier(houseprice[x],continuous=True)[1])
numerical_columns_summary

In [12]:
print(f"""{len(numerical_columns_summary)} Numerical columns with

Missing Records   : {numerical_columns_summary[numerical_columns_summary['count']<len(houseprice)].index.tolist()}
Zero units        : {numerical_columns_summary[numerical_columns_summary['min']==0].index.tolist()}
Skew > 5          : {numerical_columns_summary[numerical_columns_summary['skew']>5].index.tolist()}
Possible outliers : {numerical_columns_summary[numerical_columns_summary['max']>numerical_columns_summary['mean'] + 3*numerical_columns_summary['std']].index.tolist()}
""")

for col in ['LotArea','GrLivArea']:
    print(f"""{col} Summary:
---------------------------
Average {col} for the dataset {numerical_columns_summary.loc[col]['mean']} ft^2 with standard deviation {numerical_columns_summary.loc[col]['std']} ft^2
The {col} ranges from {numerical_columns_summary.loc[col]['min']} to {numerical_columns_summary.loc[col]['max']} with median of {numerical_columns_summary.loc[col]['50%']}
Cutoff Max for outlier removal: {data_range_nooutlier(houseprice[col],continuous=True)[1]:.2f}
""")

#### 1.3.3 Numerical-Discrete columns

In [13]:
numerical_discrete_summary = houseprice[numerical_discrete].applymap(str).describe().transpose()
numerical_discrete_summary['count'] = numerical_discrete_summary.index.map(lambda x: len(houseprice[~houseprice[x].isna()]))
numerical_discrete_summary['top'] = numerical_discrete_summary.apply(lambda x: house_mapping[x.name][x['top']] if x.name in house_mapping and house_mapping[x.name]  else x['top'] , axis=1)
numerical_discrete_summary

In [14]:
print(f"""{len(numerical_discrete_summary)} Numerical-Discrete columns with

Missing Records   : {numerical_discrete_summary[numerical_discrete_summary['count']<len(houseprice)].index.tolist()}
""")

for col in ['MSSubClass','OverallQual','GarageCars']:
    print(f"""{col} Summary:
---------------------------
Number for Unique Values  : {numerical_discrete_summary.loc[col, 'unique']}
Value with most occurances: {numerical_discrete_summary.loc[col, 'top']} ({numerical_discrete_summary.loc[col, 'freq']})

""")

#### 1.3.4 Time columns

In [15]:
time_columns_summary = pd.DataFrame(index=time_columns)
time_columns_summary['count'] = time_columns_summary.index.map(lambda x: len(houseprice[~houseprice[x].isna()])).astype("int")
time_columns_summary['unique'] = time_columns_summary.index.map(lambda x: houseprice[x].nunique()).astype("int")
time_columns_summary['min'] = time_columns_summary.index.map(lambda x: houseprice[x].min()).astype("int")
time_columns_summary['max'] = time_columns_summary.index.map(lambda x: houseprice[x].max()).astype("int")
time_columns_summary['top'] = time_columns_summary.index.map(lambda x: houseprice[x].value_counts().idxmax()).astype("int")
time_columns_summary['freq'] = time_columns_summary.index.map(lambda x: houseprice[x].value_counts().max()).astype("int")
time_columns_summary

In [16]:
print(f"""{len(time_columns_summary)} Time columns with

Missing Records   : {time_columns_summary[time_columns_summary['count']<len(houseprice)].index.tolist()}
""")

for col in ['YearBuilt','YearRemodAdd','YrSold','GarageYrBlt']:
    print(f"""{col} Summary:
---------------------------
Number for Unique Values  : {time_columns_summary.loc[col, 'unique']}
Value with most occurances: {time_columns_summary.loc[col, 'top']} ({time_columns_summary.loc[col, 'freq']})
Range                     : {time_columns_summary.loc[col, 'min']} - {time_columns_summary.loc[col, 'max']} ({time_columns_summary.loc[col, 'max'] - time_columns_summary.loc[col, 'min']})

""")

#### 1.3.5 Categorical Attributes

In [17]:
cat_summary = houseprice.describe(include=[object]).transpose()
cat_summary['top'] = cat_summary.apply(lambda x: house_mapping[x.name][x['top']] if x.name in house_mapping and x['top'] in house_mapping[x.name]  else x['top'] , axis=1)
cat_summary

In [18]:
print(f"""{len(cat_summary)} Categorical columns with

Missing Records   : {cat_summary[cat_summary['count']<len(houseprice)].index.tolist()}
""")

for col in ['MSZoning','SaleCondition','SaleCondition','Electrical','Heating','FireplaceQu']:
    print(f"""{col} Summary:
---------------------------
Number for Unique Values  : {cat_summary.loc[col, 'unique']}
Value with most occurances: {cat_summary.loc[col, 'top']} ({cat_summary.loc[col, 'freq']})

""")

### 1.4 Data Cleaning

#### 1.4.1 Dealing with Missing Value

In [19]:
# Getting the number of missing values in each column
num_missing = houseprice.isna().sum()
# Excluding columns that contains 0 missing values
num_missing = num_missing[num_missing > 0]
# Getting the percentages of missing values
percent_missing = num_missing * 100 / houseprice.shape[0]
# Concatenating the number and perecentage of missing values
# into one dataframe and sorting it
pd.concat([num_missing, percent_missing], axis=1,
keys=['Missing Values', 'Percentage']).\
sort_values(by="Missing Values", ascending=False).T.style.background_gradient(axis=1) 

In [20]:

# Getting the number of missing values in each column
num_missing = houseprice_test.isna().sum()
# Excluding columns that contains 0 missing values
num_missing = num_missing[num_missing > 0]
# Getting the percentages of missing values
percent_missing = num_missing * 100 / houseprice_test.shape[0]
# Concatenating the number and perecentage of missing values
# into one dataframe and sorting it
pd.concat([num_missing, percent_missing], axis=1,
keys=['Missing Values', 'Percentage']).\
sort_values(by="Missing Values", ascending=False).T.style.background_gradient(axis=1) 

##### 1.4.1.1 PoolQC 
Pool quality has 99.5 % missing values, which is very high. 

In [21]:
houseprice.groupby( ['PoolQC'],as_index=False, dropna=False)['PoolArea'].min().T

Pool Quality is NaN when Pool Area is 0. So we can impute the value with NA = No Pool

In [22]:
#NA = No Pool as per data description
houseprice["PoolQC"].fillna("NA", inplace=True)
houseprice_test["PoolQC"].fillna("NA", inplace=True)

##### 1.4.1.2 MiscFeature
Miscellaneous feature not covered in other categories- has ~96% null values

In [23]:
houseprice["MiscFeature"].value_counts(dropna=False).to_frame().T


In [24]:
#NA = None as per data description
houseprice["MiscFeature"].fillna("NA", inplace=True)
houseprice_test["MiscFeature"].fillna("NA", inplace=True)

##### 1.4.1.3 Alley
Type of alley access to property- has ~93% null values.

Setting the null values as No Alley

In [25]:
#NA = No Alley
houseprice['Alley'].fillna('NA', inplace=True)
houseprice_test['Alley'].fillna('NA', inplace=True)
houseprice["Alley"].value_counts(dropna=False).to_frame().T

##### 1.4.1.4 Fence
Fence quality- has ~80% null values.

Setting the null values as No Fence

In [26]:
#NA = No Fence
houseprice['Fence'].fillna('NA', inplace=True)
houseprice_test['Fence'].fillna('NA', inplace=True)
houseprice["Fence"].value_counts(dropna=False).to_frame().T

##### 1.4.1.5 FireplaceQu
Fireplace quality- has ~47% null values.


In [27]:
houseprice.groupby( ['FireplaceQu'],as_index=False, dropna=False)['Fireplaces'].min().T

Fireplace is 0 where FireplaceQu is Nan, so setting as No Fireplace

In [28]:
houseprice['FireplaceQu'].fillna('NA', inplace=True)
houseprice_test['FireplaceQu'].fillna('NA', inplace=True)

##### 1.4.1.6 LotFrontage
Linear feet of street connected to property- has ~18% null values.

We assume that the missing values in this column indicates that the house
is not connected to any street, and we fill in the missing values with 0

In [29]:
houseprice['LotFrontage'].fillna(0, inplace=True)
houseprice_test['LotFrontage'].fillna(0, inplace=True)

##### 1.4.1.7 GarageType, GarageYrBlt, GarageFinish, GarageQual, GarageCond
has ~5.6% null values.

- GarageType: Garage location
- GarageYrBlt: Year garage was built
- GarageFinish: Interior finish of the garage
- GarageQual: Garage quality
- GarageCond: Garage condition

In [30]:
#getting the records where garage attributes has null
garage_columns = [col for col in houseprice.columns if col.startswith("Garage")]
houseprice[houseprice[garage_columns].isna().any(axis=1)][garage_columns].drop_duplicates()

**When the GarageCars is 0, the other values are NaN and Area is 0**

- Setting the values to No Garage

In [31]:
for col in ['GarageType','GarageFinish','GarageQual','GarageCond']:
    houseprice[col].fillna('NA', inplace=True)
    houseprice_test[col].fillna('NA', inplace=True)

In [32]:
# Setting build year to 0
houseprice['GarageYrBlt'].fillna(0, inplace=True)
houseprice_test['GarageYrBlt'].fillna(0, inplace=True)

##### 1.4.1.8 BsmtExposure, BsmtFinType2, BsmtFinType1, BsmtCond, BsmtQual

has ~2.6% null values.

- BsmtExposure: Refers to walkout or garden level walls
- BsmtFinType1: Rating of basement finished area
- BsmtFinType2: Rating of basement finished area (if multiple types)
- BsmtCond: Evaluates the general condition of the basement
- BsmtQual: Evaluates the height of the basement


In [33]:
bsmt_columns = [col for col in houseprice.columns if col.startswith("Bsmt")]
houseprice[houseprice[bsmt_columns].isna().any(axis=1)][bsmt_columns].drop_duplicates()

Three types of data
- Bsmt data is either Nan or 0 : Marking those entries as No Basement
- BsmtConf as TA and BsmtFinType2 as Nan : set as Unf
- BsmtConf as TA and BsmtExposure as Nan : set as No

In [34]:
# Setting the values as NA as `Expose is Nan or No
#Case 1
houseprice.loc[~pd.isnull(houseprice['BsmtCond']) & pd.isnull(houseprice['BsmtFinType2']),'BsmtFinType2']='Unf'
#case 2
houseprice.loc[~pd.isnull(houseprice['BsmtCond']) & pd.isnull(houseprice['BsmtExposure']),'BsmtExposure']='No'
#Case 3
for col in ["BsmtExposure", "BsmtFinType2", "BsmtFinType1", "BsmtCond", "BsmtQual"]:
    houseprice[col].fillna('NA', inplace=True)
    houseprice_test[col].fillna('NA', inplace=True)

##### 1.4.1.9 MasVnrArea, MasVnrType

has ~0.5% null values.

- MasVnrType: Masonry veneer type
- MasVnrArea: Masonry veneer area in square feet

In [35]:
# set as 0 and None as per data definition
houseprice['MasVnrArea'].fillna(0, inplace=True)
houseprice_test['MasVnrArea'].fillna(0, inplace=True)
houseprice['MasVnrType'].fillna("None", inplace=True)
houseprice_test['MasVnrType'].fillna("None", inplace=True)

##### 1.4.1.10 Electrical

Electrical: Electrical system has 1 null value.

In [36]:
## Getting the Electrical used during the year build
print(" Record is null when \n ",houseprice.loc[houseprice['Electrical'].isna(),['YearBuilt','YearRemodAdd']])
print("\n Getting the Electrical used during the period: \n", houseprice.loc[((houseprice['YearBuilt']==2006)  | (houseprice['YearRemodAdd']==2007)),'Electrical'].value_counts())

**Standard Circuit Breakers & Romex** was used during the year build 2006, so setting the Electrical value 

In [37]:
houseprice['Electrical'].fillna("SBrkr", inplace=True)
houseprice_test['Electrical'].fillna("SBrkr", inplace=True)

In [38]:
# Getting the number of missing values in each column
num_missing = houseprice_test.isna().sum()
# Excluding columns that contains 0 missing values
num_missing = num_missing[num_missing > 0]
# Getting the percentages of missing values
percent_missing = num_missing * 100 / houseprice_test.shape[0]
# Concatenating the number and perecentage of missing values
# into one dataframe and sorting it
pd.concat([num_missing, percent_missing], axis=1,
keys=['Missing Values', 'Percentage']).\
sort_values(by="Missing Values", ascending=False).T.style.background_gradient(axis=1) 


In [39]:
houseprice_test.loc[houseprice_test.isna().any(axis=1),
                    num_missing.index.tolist()]
houseprice_test['MSZoning'].fillna("RL", inplace=True)
houseprice_test['Utilities'].fillna("AllPub", inplace=True)
houseprice_test['Exterior1st'].fillna('VinylSd', inplace=True)
houseprice_test['BsmtFinSF2'].fillna(0.00, inplace=True)
houseprice_test['BsmtUnfSF'].fillna(0.00, inplace=True)
houseprice_test['TotalBsmtSF'].fillna(0.00, inplace=True)
houseprice_test['BsmtFullBath'].fillna(0.00, inplace=True)
houseprice_test['BsmtHalfBath'].fillna(0.00, inplace=True)
houseprice_test['KitchenQual'].fillna('TA', inplace=True)
houseprice_test['Functional'].fillna('Typ', inplace=True)
houseprice_test['SaleType'].fillna('WD', inplace=True)
houseprice_test['GarageCars'].fillna(0.00, inplace=True)
houseprice_test['GarageArea'].fillna(0.00, inplace=True)

### 1.5 Outliers

- There seems to be **outliers**, as max values more than double the 3rd quartile 
- There are features with 75 percentile value as 0 and values above 3rd quater

In [40]:

def plot_hist_scatter(df, feature, target="SalePrice", outlier=True, log=False, **kwargs):
    if log:
        feature_data = np.log1p(df[feature])
    else:
        feature_data = df[feature]
    plt.figure(figsize=(2 * 5.2, 1 * 3.2))
    
    print(f"skew={feature_data.skew()}, kurtosis={feature_data.kurtosis()}")
    
    plt.subplot(1, 2, 1)
    feature_data.hist(bins=50)
    if outlier:
        cuttoff_min, cuttoff_max = data_range_nooutlier(feature_data, log=False, **kwargs)
        print(f"Within range: between {cuttoff_min:.2f} and {cuttoff_max:.2f}")
        plt.axvspan(xmin=cuttoff_max, xmax=feature_data.max(), color="r", alpha=0.5)
    plt.xlabel(f"{'log' if log else ''}{feature}")
    plt.ylabel("count")
    plt.title(f"{'log' if log else ''}{feature} histogram")
    if feature != target:
        plt.subplot(1, 2, 2)
        plt.scatter(
            x=feature_data,
            y=df[target],
            color="orange",
            edgecolors="#000000",
            linewidths=0.5,
        )
        plt.xlabel(feature)
        plt.ylabel(f"{target}")
        if outlier:
            plt.axvspan(xmin=cuttoff_max, xmax=feature_data.max(), color="r", alpha=0.5)
    plt.show()


# plotting bars
def plot_bar(df, columns):
    cols = 3
    rows = len(columns) // 3 + 1

    plt.figure(figsize=(cols * 6.7, rows * 3.75))
    i = 0
    for row in range(rows):
        for col in range(cols):
            index = cols * row + col
            if index >= len(columns):
                break
            plt.subplot(rows, cols, index + 1)
            df.groupby(columns[i]).size().plot(
                kind="bar",
            )
            
            i += 1


# plotting box
def plot_box(df, y, columns):
    cols = 3
    rows = len(columns) // 3 + 1

    plt.figure(figsize=(cols * 5.5, rows * 3.5))
    i = 0
    for row in range(rows):
        for col in range(cols):
            index = cols * row + col
            if index >= len(columns):
                break
            plt.subplot(rows, cols, index + 1)
            sns.boxplot(x=columns[i], y=y, data=df)

            i += 1

# plotting hist
def plot_hist(df, columns):
    cols = 3
    rows = len(columns) // 3 + 1

    plt.figure(figsize=(cols * 5.5, rows * 3.5))
    i = 0
    for row in range(rows):
        for col in range(cols):
            index = cols * row + col
            if index >= len(columns):
                break
            plt.subplot(rows, cols, index + 1)
            df[columns[i]].hist(bins=50)
            plt.ylabel(columns[i])

            i += 1


#plot scatter            
def plot_scatter(df,y, columns):
    cols = 3
    rows = len(columns) // 3 + 1

    plt.figure(figsize=(cols * 7.5, rows * 5.2))
    i = 0
    for row in range(rows):
        for col in range(cols):
            index = cols * row + col
            if index >= len(columns):
                break
            plt.subplot(rows, cols, index + 1)
            sns.scatterplot(x=columns[i], y=y, data=df)

            i += 1

            
def calculateAnova(inpData,y, catCols, target):
    inpData = inpData.join(y)
    from scipy.stats import f_oneway
    CatColumnList = []
    for cat in catCols:
        CatGroupList = inpData.groupby(cat)[target].apply(list)
        anova = f_oneway(*CatGroupList)
        if(anova[1]<0.05):
            print('The column ', cat, ' is correlated with ', target, ' | P-Value: ',anova[1])
            CatColumnList.append(cat)
        else:
            print('The column ', cat , ' is NOT correlated with ', target, ' | P-Value: ',anova[1])
    
    return(CatColumnList)

In [41]:
cuttoff = dict()
for col in numerical_columns:
    cuttoff[col] = dict()
    cuttoff[col]['min'], cuttoff[col]['max']= houseprice[col].min(),houseprice[col].max()
    cuttoff[col]['cutoffmin'],cuttoff[col]['cutoffmax']=data_range_nooutlier(houseprice[col],continuous=True)
    cuttoff[col]['p25'], cuttoff[col]['p75']= np.percentile(houseprice[col],[25,75])
    cuttoff[col]['cutoffmin'] = cuttoff[col]['cutoffmin'] if cuttoff[col]['min'] < cuttoff[col]['cutoffmin'] else  cuttoff[col]['min']
    plot_hist_scatter(houseprice,col, continuous=True)
    

In [42]:
pd.DataFrame.from_dict(cuttoff).T[['min','cutoffmin','p25','p75','cutoffmax','max']]

In [43]:
## Finding the columns having High corelation with SalePrice
cor_df =houseprice[numerical_columns + ['SalePrice']].corr()['SalePrice'].sort_values(key=abs, ascending=False).to_frame()
cor_df.style.background_gradient(axis=1)

In [44]:
## Removing outliers for features having high correlation
# Commented to validate the Outlier during EDA
# adding a threshold for cutoffmax
threshold_ratio = 1.2
for col in cor_df[abs(cor_df['SalePrice']) >= 0.5].index.tolist():
    if col != 'SalePrice':
        print(f"Adding cuttoff max for {col} : {cuttoff[col]['cutoffmax'] *threshold_ratio:.2f}")
        houseprice = houseprice[houseprice[col] <=cuttoff[col]['cutoffmax']*threshold_ratio]
print("Shape after removing outlier ", houseprice.shape)

## 2. Exploratory Data Analysis

In [45]:
X = houseprice.copy()
X_test = houseprice_test.copy()
y = X.pop("SalePrice")

### 2.1 Univariate Analysis

#### 2.1.1 SalePrice

In [46]:
fig, axes = plt.subplots(1, 2, figsize=(15, 3), sharey=False)
fig.suptitle('Histogram')


sns.histplot(data=y, kde=True, ax=axes[0])
axes[0].set_title('SalePrice ')


sns.histplot(ax=axes[1], data=np.log(y), kde=True)
axes[1].set_title('Log of SalePrice')
axes[1].set_xlabel('Log SalePrice')

plt.show()

- Sales price is positively skewed, and 
- **Log of data resembles a normal distribution**

In [47]:
# Sales per Square Feet
SalePriceSF = y/X['GrLivArea']
plt.hist(SalePriceSF, bins=15,color="gold")
plt.title("Sale Price per Square Foot")
plt.ylabel('Number of Sales')
plt.xlabel('Price per square feet');
#Average Sale Price per square feet 
print("Average Sale Price per square feet: $",SalePriceSF.mean())

#### 2.1.2 Ordinal and Nominal

In [48]:
cat_columns=cat_summary.index.tolist()

In [49]:
plot_bar(X, numerical_discrete+ time_columns + cat_columns)

#### 2.1.3 Numerical

In [50]:
plot_hist(X,numerical_columns)

### 2.2 Multi variate Analysis

#### 2.2.1 Correlation

In [51]:
X.columns

In [52]:
plot_box(X,y,numerical_discrete+ time_columns + cat_columns)

From the above Plot
- Utilities doesnot show any corelation with Sales Price and most of the data are AllPub
- Having AC definitely escalates price of house.
- Having 1 Kitchen of Excellent quality hikes house price like anything.

In [53]:
fig, ax = plt.subplots(figsize=(20,7))
sns.heatmap(X[numerical_columns].corr(), cmap='coolwarm', annot=True, annot_kws={'size':10}, )
plt.show()

- GrLivArea ,GarageCars,GarageArea ,TotalBsmtSF, 1stFlrSF  have more than 0.5 correlation with SalePrice.
- EnclosedPorch and KitchenAbvGr have little negative correlation with target variable.

In [54]:
correlations=X.corr()
attrs = correlations.iloc[:-1,:-1] # all except target

threshold = 0.5
important_corrs = (attrs[abs(attrs) > threshold][attrs != 1.0]) \
    .unstack().dropna().to_dict()

unique_important_corrs = pd.DataFrame(
    list(set([(tuple(sorted(key)), important_corrs[key]) \
    for key in important_corrs])), 
        columns=['Attribute Pair', 'Correlation'])

    # sorted by absolute value
unique_important_corrs = unique_important_corrs.iloc[
    abs(unique_important_corrs['Correlation']).argsort()[::-1]]

unique_important_corrs.style.background_gradient(axis=0)

This shows multicollinearity. In regression, "multicollinearity" refers to features that are correlated with other features. Multicollinearity occurs when your model includes multiple factors that are correlated not just to your target variable, but also to each other.

To avoid this we can do 3 things:

- Completely remove those variables
- Make new feature by adding them or by some other operation.
- Use PCA, which will reduce feature set to small number of non-collinear features.

#### 2.2.2 Visualization

##### 2.2.2.1 GarageArea - Sales Price

Since GarageArea is highly correlated with the target variable, let's focus on it first.

In [55]:
sns.scatterplot(y=y, x = 'GarageArea', data=X)
plt.show()

The above plot clearly shows a linear relationship between ‘SalePrice’ and ‘GarageArea’. The ‘SalePrice’ increases with an increase in ‘GarageArea’.

##### 2.2.2.2 OverallQual - Sales Price

OverallQual is highly correlated with the target variable

In [56]:
sns.regplot(x='OverallQual', y=y, data=X, robust=True)

Obviously, there is a linear relationship between OverallQual and SalePri

##### 2.2.2.3 YearBuilt and SalePrice

In [57]:
sns.regplot(y=y, x='YearBuilt', data=X)

‘YearBuilt’ shows that the distribution is skewed towards the year 2000 and has a long tail which extends till 1900.
The linear relationship between the variables is clearer in cases of recently built houses.

##### 2.2.2.3 TotalBsmtSF and SalePrice

In [58]:
sns.regplot(x='TotalBsmtSF', y=y, data=X)

TotalBsmtSF is very highly correlated with our target variable SalePrice and also it follows a strong linear trend.

##### 2.2.2.3 GarageYrBlt and SalePrice

In [59]:
sns.regplot(y=y, x='GarageYrBlt', data=X)

In [60]:
X_test[X_test.isna().any(axis=1)]

## 3. Feature Engineering

### 3.1 Feature Addition

In [61]:
X["LivLotRatio"] = X.GrLivArea / (X.LotArea + 1) 
X_test["LivLotRatio"] = X_test.GrLivArea / (X.LotArea + 1)
X["Spaciousness"] = (X["1stFlrSF"] + X["2ndFlrSF"]) / (X.TotRmsAbvGrd+1)
X_test["Spaciousness"] = (X_test["1stFlrSF"] + X_test["2ndFlrSF"]) / (X_test.TotRmsAbvGrd +1)
X["MedNhbdArea"] = X.groupby("Neighborhood")["GrLivArea"].transform("median")
X_test["MedNhbdArea"] = X_test.groupby("Neighborhood")["GrLivArea"].transform("median")
X["PorchTypes"] = X[[
        "WoodDeckSF",
        "OpenPorchSF",
        "EnclosedPorch",
        "3SsnPorch",
        "ScreenPorch",
    ]].gt(0.0).sum(axis=1)
X_test["PorchTypes"] = X_test[[
        "WoodDeckSF",
        "OpenPorchSF",
        "EnclosedPorch",
        "3SsnPorch",
        "ScreenPorch",
    ]].gt(0.0).sum(axis=1)

### 3.2 Feature Selection

#### 3.2.1 Continuous Vs Continuous -- Scatter Chart

In [62]:
numerical_columns=numerical_columns+['LivLotRatio','Spaciousness','MedNhbdArea']

In [63]:
plot_scatter(X,y, numerical_columns)

Based on correction, selecting the continous variables with correction > 0.4

In [64]:
corr_mat = X[numerical_columns].join(y).corr()
selected_numerical_columns = corr_mat['SalePrice'][abs(corr_mat['SalePrice'])>=0.5].index.tolist()
nonselected_numerical_columns = corr_mat['SalePrice'][abs(corr_mat['SalePrice'])<0.5].index.tolist()
selected_numerical_columns

#### 3.2.2 Categorical Vs Continuous -- Anova

In [65]:
#target = 'SalePrice'
selected_categorical_cols = calculateAnova(X,y, numerical_discrete+ nonselected_numerical_columns + time_columns + cat_columns,'SalePrice')
selected_categorical_cols

In [66]:
selected_col = [col for col in X.columns \
 if col in selected_numerical_columns + selected_categorical_cols]
X = X[selected_col].copy()
X_test = X_test[selected_col].copy()

#### 3.2.3 Cross correlation between attributes

In [67]:
correlations=X.corr()
attrs = correlations.iloc[:-1,:-1] # all except target

threshold = 0.5
important_corrs = (attrs[abs(attrs) > threshold][attrs != 1.0]) \
    .unstack().dropna().to_dict()

unique_important_corrs = pd.DataFrame(
    list(set([(tuple(sorted(key)), important_corrs[key]) \
    for key in important_corrs])), 
        columns=['Attribute Pair', 'Correlation'])

    # sorted by absolute value
unique_important_corrs = unique_important_corrs.iloc[
    abs(unique_important_corrs['Correlation']).argsort()[::-1]]

unique_important_corrs.style.background_gradient(axis=0)

To avoid the Multicollinearity problem, we will delete one feature from each pair of highly
correlated predictors. We have two pairs: the first consists of Garage Cars and Garage Area, and
the other consists of Gr Liv Area and TotRms AbvGrd. For the first pair, we will remove Garage
Cars feature; from the second pair, we will remove TotRms AbvGrd feature:

In [68]:
X.drop(["GarageCars", "GarageYrBlt","TotRmsAbvGrd"], axis=1, inplace=True)
X_test.drop(["GarageCars", "GarageYrBlt","TotRmsAbvGrd"], axis=1, inplace=True)

#### 3.2.3 mutual_info_regression

In [69]:
def make_mi_scores(X, y):
    '''Estimate mutual information for a continuous target variable.'''
    X = X.copy()
    for colname in X.select_dtypes(["object", "category"]):
        X[colname], _ = X[colname].factorize()
    # All discrete features should now have integer dtypes
    discrete_features = [pd.api.types.is_integer_dtype(t) for t in X.dtypes]
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features, random_state=0)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores
mi_scores = make_mi_scores(X, y)
mi_scores.head()
def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.figure(figsize=(10,15))
    clrs = ['grey' if (x < 0.01) else 'blue' for x in scores ]
    plt.barh(width, scores, color=clrs)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")
plot_mi_scores(mi_scores)

In [70]:
def corrplot(df, method="pearson", annot=False, **kwargs):
    plt.figure(figsize=(25,40))
    sns.clustermap(
        df.corr(method),
        vmin=-1.0,
        vmax=1.0,
        cmap="icefire",
        method="complete",
        annot=annot,
        **kwargs,
    )
corrplot(X.join(y))

### 3.3  Feature Scaling and Transform

In [71]:
from sklearn.preprocessing import StandardScaler
def transform_dataset(dataset, test_data):

    def transform_categoricals(dataset):
        mp = {'Ex':4,'Gd':3,'TA':2,'Fa':1,'Po':0}
        dataset['ExterQual'] = dataset['ExterQual'].map(mp)
        #dataset['ExterCond'] = dataset['ExterCond'].map(mp)
        dataset['HeatingQC'] = dataset['HeatingQC'].map(mp)
        dataset['KitchenQual'] = dataset['KitchenQual'].map(mp)
        mp = {'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'NA':0}
        dataset['BsmtQual'] = dataset['BsmtQual'].map(mp)
        dataset['BsmtCond'] = dataset['BsmtCond'].map(mp)
        dataset['BsmtExposure'] = dataset['BsmtExposure'].map(
        {'Gd':4,'Av':3,'Mn':2,'No':1,'NA':0})
        mp = {'GLQ':6,'ALQ':5,'BLQ':4,'Rec':3,'LwQ':2,'Unf':1,'NA':0}
        dataset['BsmtFinType1'] = dataset['BsmtFinType1'].map(mp)
        dataset['BsmtFinType2'] = dataset['BsmtFinType2'].map(mp)
        dataset['CentralAir'] = dataset['CentralAir'].map({'Y':1,'N':0})
        dataset['Functional'] = dataset['Functional'].map(
        {'Typ':7,'Min1':6,'Min2':5,'Mod':4,'Maj1':3,
        'Maj2':2,'Sev':1,'Sal':0})
        dataset['FireplaceQu'] = dataset['FireplaceQu'].map(
        {'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'NA':0})
        dataset['GarageFinish'] = dataset['GarageFinish'].map(
        {'Fin':3,'RFn':2,'Unf':1,'NA':0})
        dataset['GarageQual'] = dataset['GarageQual'].map(
        {'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'NA':0})
        dataset['GarageCond'] = dataset['GarageCond'].map(
        {'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'NA':0})
        dataset['Fence'] = dataset['Fence'].map(
        {'GdPrv':4,'MnPrv':3,'GdWo':2,'MnWw':1,'NA':0})
        return dataset
    
    numerical_columns = dataset.select_dtypes(include=[np.number]).columns.tolist()
    dataset = transform_categoricals(dataset)
    test_data = transform_categoricals(test_data)
    scaler = StandardScaler()
    # We need to fit the scaler to our data before transformation
    dataset.loc[:, numerical_columns] = scaler.fit_transform(
    dataset.loc[:, numerical_columns])
    test_data.loc[:, numerical_columns] = scaler.transform(
    test_data.loc[:, numerical_columns])
    
    cat_cols = dataset.select_dtypes(exclude=[np.number]).columns
    
    # Removing uninformative feed
    dataset = dataset.loc[:,mi_scores>=0.05]
    test_data = test_data.loc[:,mi_scores>=0.05]
    dataset = pd.get_dummies(dataset)
    test_data = pd.get_dummies(test_data)

    return dataset, test_data

In [72]:
X, X_test=transform_dataset(X, X_test)

## 4. Training Model

In [73]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVR

from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

#### 4.1 Splitting the Dataset<a class="anchor" id="tr_split">
    
We need a training dataset to train our model
and a test dataset to evaluate the model. So we will split our dataset randomly into two parts,
one for training and the other for testing

In [74]:

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y,
test_size=0.3, random_state=3)


#### 4.2 Training<a class="anchor" id="tr_apr">
    
Choose an algorithm that implements the corresponding technique
    
- Search for an effective parameter combination for the chosen algorithm
- Create a model using the found parameters
- Train (fit) the model on the training dataset
- Test the model on the test dataset and get the results

##### 4.2.1 Linear Regression<a class="anchor" id="tr_lr">
For Linear Regression, we will choose three algorithmic implementations: Ridge Regression and
Elastic Net.

###### 4.2.1.1 Ridge Regression

In [75]:
parameter_space = {
"alpha": [1, 10, 100, 290, 500],
"fit_intercept": [True, False],
"solver": ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
}
clf = GridSearchCV(Ridge(random_state=3), parameter_space, n_jobs=-1,
cv=5, scoring="neg_mean_squared_error")
clf.fit(Xtrain, ytrain)
print("Best parameters:")
print(clf.best_params_)

ridge_model = Ridge(random_state=3, **clf.best_params_)
ridge_model.fit(Xtrain, ytrain);

ypred = ridge_model.predict(Xtest)
ridge_mse = mean_squared_error(ytest, ypred)
print("Ridge MSE =", ridge_mse)

In [76]:
parameter_space = {
"alpha": [1, 10, 100, 280, 500],
"l1_ratio": [0.5, 1],
"fit_intercept": [True, False],
}
clf = GridSearchCV(ElasticNet(random_state=3), parameter_space,
n_jobs=-1, cv=5, scoring="neg_mean_squared_error")
clf.fit(Xtrain, ytrain)
clf.fit(Xtest, ytest)
print("Best parameters:")
print(clf.best_params_)
elasticNet_model = ElasticNet(random_state=3, **clf.best_params_)
elasticNet_model.fit(Xtrain, ytrain)
ypred = elasticNet_model.predict(Xtest)
elasticNet_mse = mean_squared_error(ytest, ypred)
print("Elastic Net MSE =", elasticNet_mse)

In [77]:
parameter_space = \
{
"max_depth": [4, 5, 6],
"learning_rate": [0.005, 0.009, 0.01],
"n_estimators": [700, 1000, 2500],
"booster": ["gbtree",],
"gamma": [7, 25, 100],
"subsample": [0.3, 0.6],
"colsample_bytree": [0.5, 0.7],
"colsample_bylevel": [0.5, 0.7,],
"reg_alpha": [0.5, 1, 10, 33],
"reg_lambda": [1, 3, 10],
}
clf = RandomizedSearchCV(XGBRegressor(random_state=3),
parameter_space, cv=10, n_jobs=-1,
scoring="neg_mean_squared_error",
random_state=3, n_iter=20)
clf.fit(X, y)
print("Best parameters:")
print(clf.best_params_)
xgb_model = XGBRegressor(**clf.best_params_)
xgb_model.fit(Xtrain, ytrain);
y_pred = xgb_model.predict(Xtest)
xgb_mse = mean_squared_error(ytest, y_pred)
print("XGBoost MSE =", xgb_mse)

In [78]:
xgb_feature_importances = xgb_model.feature_importances_
xgb_feature_importances = pd.Series(
xgb_feature_importances, index=Xtrain.columns.values
).sort_values(ascending=False).head(15)
fig, ax = plt.subplots(figsize=(7, 5))
sns.barplot(x=xgb_feature_importances,
y=xgb_feature_importances.index,
color="#003f5c");
plt.xlabel('Feature Importance');
plt.ylabel('Feature');

In [79]:
for x in [x for x in Xtrain.columns
 if x not in X_test.columns]:
    X_test[x]= 0

In [81]:
predictions

In [82]:
xgb_model

In [93]:
import pickle 
model_file = "model.pickle"
with open(model_file,'wb') as f:
    pickle.dump(xgb_model, f)

In [80]:

predictions = xgb_model.predict(X_test)
output = pd.DataFrame({'Id': X_test.index, 'SalePrice': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")